In [ ]:
pip install -U peft

In [ ]:
pip install trl

In [ ]:
pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
pip install -U accelerate

In [1]:
import numpy as np
import pandas as pd


import torch
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
import transformers
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments,
                          )
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer

2024-05-28 21:03:17.902046: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 21:03:17.902106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 21:03:17.903591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
model_name = "google/gemma-2b-it"

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)
model.config.use_cache = False
model.config.pretraining_tp = 1
max_seq_length = 1024
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def question_gemma(question, model=model, tokenizer=tokenizer, temperature=0.0, return_answer=False):
    input_ids = tokenizer(question, return_tensors="pt").to("cuda")
    if temperature > 0:
        do_sample=True
    else:
        do_sample=False
    outputs = model.generate(**input_ids, 
                             max_new_tokens=256, 
                             do_sample=do_sample, 
                             temperature=temperature)
    result = str(tokenizer.decode(outputs[0])).replace("<bos>", "").replace("<eos>", "").strip()
    if return_answer:
        return result
    else:
        print(result)

In [5]:
question_gemma("C'est qui le doyen de FST Tanger ?")

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


C'est qui le doyen de FST Tanger ?

Le doyen de FST Tanger est un homme politique et syndicaliste belge. Il est le leader du Parti socialiste et des travailleurs (PS).


In [6]:
# Load dataset from CSV
df = pd.read_csv('/kaggle/input/feqqqqqqqqq/FSTT_FT_DATA.csv')

In [7]:
qa_data = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    question = row['Question']
    answer = row['Answer']
    context=row['Context']
    qa_data.append(f"{question}\n{answer} {context}")

In [8]:
qa_data

["Quels sont les objectifs de formation du tronc commun MIP ?\nL'objectif du tronc commun MIP est de donner à l’étudiant une base solide dans les matières scientifiques (physique, chimie et mathématiques etc…) Les objectifs de formation - Tronc commun MIP",
 "Qu'est-ce que le tronc commun MIP permet à l’étudiant de faire ?\nLe tronc commun MIP permet à l’étudiant de continuer ses études dans les semestres S5 et S6 des cycles licences qui émanent de ce parcours tout en gardant la possibilité de se réorienter vers d’autres troncs communs comme MIPC et GE/GM. Les objectifs de formation - Tronc commun MIP",
 'Quelles sont les matières abordées dans le tronc commun MIP ?\nLes matières abordées dans le tronc commun MIP sont la physique, la chimie et les mathématiques. Les objectifs de formation - Tronc commun MIP',
 "Comment les étudiants peuvent-ils utiliser le tronc commun MIP pour postuler à des concours ?\nLe tronc commun MIP permet aux étudiants de postuler à des concours d’accès aux cy

In [9]:
max_seq_length = 1024

data = (pd.DataFrame(qa_data, columns=["text"])
              .sample(frac=1, random_state=5)
              .drop_duplicates()
             )
data = Dataset.from_pandas(data)

In [10]:
data

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 707
})

In [11]:
output_dir = "./results"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    evaluation_strategy='steps',
    eval_steps = 500,
    eval_accumulation_steps=1,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
)

In [12]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/707 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=440, training_loss=0.5730119233781641, metrics={'train_runtime': 1114.6379, 'train_samples_per_second': 3.171, 'train_steps_per_second': 0.395, 'total_flos': 3598054935158784.0, 'train_loss': 0.5730119233781641, 'epoch': 4.98})

In [16]:
question_gemma("Quels sont les avantages des clubs des étudiants à la FST de Tanger?",
                model=model, tokenizer=tokenizer)

Quels sont les avantages des clubs des étudiants à la FST de Tanger?
Ils offrent une occasion aux étudiants de s’engager dans des activités qui les passionnent, de rencontrer des personnes partageant les mêmes intérêts, de renforcer les compétences de leadership et de développer des compétences professionnelles. Clubs des étudiants – La pratique des clubs et d’activités proposés par la FST de Tanger. S’engager dans des activités qui les passionnent et de rencontrer des personnes partageant les mêmes intérêts. Renforcer les compétences de leadership et développer des compétences professionnelles. Offrir une occasion aux étudiants de s’engager dans des activités qui les passionnent et de rencontrer des personnes partageant les mêmes intérêts. Renforcer les compétences de leadership et développer des compétences professionnelles. https://fstt.ac.ma/Portail2023/clubs-et-activites/ Les clubs des étudiants peuvent offrir une variété de activités, telles que des clubs sportifs, culturelles, p

In [17]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/tokenizer.model',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [19]:
import gc

del [model, tokenizer, peft_config, trainer, data, bnb_config, training_arguments]
del [TrainingArguments, SFTTrainer, LoraConfig, BitsAndBytesConfig]

for _ in range(10):
    torch.cuda.empty_cache()
    gc.collect()

NameError: name 'model' is not defined

In [21]:
from peft import AutoPeftModelForCausalLM

finetuned_model = output_dir
compute_dtype = getattr(torch, "float16")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoPeftModelForCausalLM.from_pretrained(
     finetuned_model,
     torch_dtype=compute_dtype,
     return_dict=False,
     low_cpu_mem_usage=True,
     device_map="auto",
)
merged_model = model.merge_and_unload()
merged_model.save_pretrained("./results_merged",
                             safe_serialization=True, 
                             max_shard_size="2GB")
tokenizer.save_pretrained("./results_merged")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('./results_merged/tokenizer_config.json',
 './results_merged/special_tokens_map.json',
 './results_merged/tokenizer.model',
 './results_merged/added_tokens.json',
 './results_merged/tokenizer.json')

In [22]:
import gc

del [model, tokenizer, merged_model, AutoPeftModelForCausalLM]
for _ in range(10):
    torch.cuda.empty_cache()
    gc.collect()

In [23]:
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig)

model_name = "results_merged"
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)
model.config.use_cache = False
model.config.pretraining_tp = 1
max_seq_length = 1024
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
question_gemma("Quels outils et technologies sont disponibles au Green Lab?",
                model=model, tokenizer=tokenizer)

Quels outils et technologies sont disponibles au Green Lab?
Des imprimantes 3D, des machines de découpe laser, des fraiseuses CNC et des équipements électroniques PRESENTATION DE LA FST DE TANGER & DIAGNOSIQUE - Le Green Lab est équipé d’une variété d’outils et de technologies, tels que des imprimantes 3D, des machines de découpe laser, des fraiseuses CNC et des équipements électroniques. Cette infrastructure permet aux étudiants de réaliser des projets de conception, de développement et de mise en œuvre d’innovations. Outillage et technologies disponibles au Green Lab à Tanger : imprimantes 3D, machines de découpe laser, fraiseuses CNC et équipements électroniques. Image de la page: https://fstt.ac.ma/Portail2023/green-lab/ Les objectifs de formation - Green Lab - Source: https://fstt.ac.ma/Portail2023/green-lab/ Les objectifs de formation de la FST de Tanger pour le Green Lab comprennent la réalisation de projets de conception, développement et mise en œuvre d’innovations, la formati

In [ ]:
# Function to generate a response from the model using an improved template
def ask_question_with_improved_template(question, model, tokenizer, max_length=200,temperature=0.0, return_answer=False):
    prompt_template = f"""
        You are an expert  AI assistant. Please answer  the following question directly and concisely, providing only the most relevant information.Answer in french language.
        Question: {question}
        Answer: """
    
    inputs = tokenizer.encode(prompt_template, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id,temperature=temperature)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the answer part from the template
    answer_start = response.find("Answer:") + len("Answer: ")
    answer = response[answer_start:].strip()
    
    return answer